# Extraction et formatage de données

### Source 1 : MusicBrainz => <a href='https://data.metabrainz.org/pub/musicbrainz/data-out/musicbrainz/fullexport/'>fichiers PostgreSQL</a>
 #### 1. Identification des fichiers utiles grâce au <a href='https://wiki.musicbrainz.org/images/4/46/entity_network_details.svg'>schéma de données MusicBrainz</a>
 #### 2. Parsing (format UTF-8)
  - ##### Récupération des tables et des index nécessaires
  - ##### Extraction des fichiers plats au format UTF-8 en fichiers csv

In [1]:
import os, csv

if not os.path.exists('data-out/musicbrainz/mb_track.csv'):
    # Open the input file
    with open('data-in/musicbrainz/track', 'r', encoding='utf-8') as tsv_in:
        print("Extracting track table...")
        # Open the output file
        with open('data-out/musicbrainz/mb_track.csv', 'w', encoding='utf-8', newline='') as csv_out:
            # Create a csv writer
            csv_writer = csv.writer(csv_out, delimiter='\t')
            csv_writer.writerow(["track_id", "artist_id", "medium_id", "track_name"])

            # Iterate over each line in the tsv file
            for line in tsv_in:
                # Split the line into columns
                columns = line.split('\t')

                # Extract the 5 columns
                track_id = columns[0]
                medium_id = columns[3]
                track_name = columns[6]
                artist_id = columns[7]

                # Write the 5 columns in the output file
                csv_writer.writerow([track_id, artist_id, medium_id, track_name])
        print("Done extracting.")

if not os.path.exists('data-out/musicbrainz/mb_artist.csv'):
    with open('data-in/musicbrainz/artist', 'r', encoding='utf-8') as tsv_in:
        print("Extracting artist table...")
        # Open the output file
        with open('data-out/musicbrainz/mb_artist.csv', 'w', encoding='utf-8', newline='') as csv_out:
            # Create a csv writer
            csv_writer = csv.writer(csv_out, delimiter='\t')
            csv_writer.writerow(["artist_id", "artist_name"])

            # Iterate over each line in the tsv file
            for line in tsv_in:
                # Split the line into columns
                columns = line.split('\t')

                # Extract the 2 columns
                artist_id = columns[0]
                artist_name = columns[2]

                # Write the 2 columns in the output file
                csv_writer.writerow([artist_id, artist_name])
        print("Done extracting.")

if not os.path.exists('data-out/musicbrainz/mb_medium.csv'):
    with open('data-in/musicbrainz/medium', 'r', encoding='utf-8') as tsv_in:
        print("Extracting medium table...")
        # Open the output file
        with open('data-out/musicbrainz/mb_medium.csv', 'w', encoding='utf-8', newline='') as csv_out:
            # Create a csv writer
            csv_writer = csv.writer(csv_out, delimiter='\t')
            csv_writer.writerow(["medium_id", "release_id"])

            # Iterate over each line in the tsv file
            for line in tsv_in:
                # Split the line into columns
                columns = line.split('\t')

                # Extract the 2 columns
                medium_id = columns[0]
                release_id = columns[1]

                # Write the 2 columns in the output file
                csv_writer.writerow([medium_id, release_id])
        print("Done extracting.")

if not os.path.exists('data-out/musicbrainz/mb_release_country.csv'):
    with open('data-in/musicbrainz/release_country', 'r', encoding='utf-8') as tsv_in:
        print("Extracting release_country table...")
        # Open the output file
        with open('data-out/musicbrainz/mb_release_country.csv', 'w', encoding='utf-8', newline='') as csv_out:
            # Create a csv writer
            csv_writer = csv.writer(csv_out, delimiter='\t')
            csv_writer.writerow(["release_id", "date_year"])

            # Iterate over each line in the tsv file
            for line in tsv_in:
                # Split the line into columns
                columns = line.split('\t')

                # Extract the 2 columns
                release_id = columns[0]
                date_year = columns[2]

                # Write the 2 columns in the output file
                csv_writer.writerow([release_id, date_year])
        print("Done extracting.")

Extracting track table...
Done extracting.
Extracting artist table...
Done extracting.
Extracting medium table...
Done extracting.
Extracting release_country table...
Done extracting.


 #### 3. Jointure pour obtenir les informations souhaitées
 - ##### track = nom du morceau
 - ##### artist = nom de l'artiste
 - ##### year = année de sortie

In [18]:
import pandas as pd

# Load the 4 csv files in 4 pandas dataframe
track_df = pd.read_csv('data-out/musicbrainz/mb_track.csv', sep='\t')
artist_df = pd.read_csv('data-out/musicbrainz/mb_artist.csv', sep='\t')
medium_df = pd.read_csv('data-out/musicbrainz/mb_medium.csv', sep='\t')
release_country_df = pd.read_csv('data-out/musicbrainz/mb_release_country.csv', sep='\t')

# Convert the 'medium_id' column to int type
medium_df['medium_id'] = medium_df['medium_id'].astype(int)
track_df['medium_id'] = track_df['medium_id'].astype(int)

# Convert the 'date_year' column to int (after removing incorrect values)
release_country_df = release_country_df[release_country_df.date_year.str.len() == 4]
release_country_df['date_year'] = release_country_df['date_year'].astype(int)

# Join the dataframes using the proper ids
df = pd.merge(track_df, artist_df, on='artist_id')
df = pd.merge(df, medium_df, on='medium_id')
df = pd.merge(df, release_country_df, on='release_id')

# Select the required columns
df = df[['track_id', 'track_name', 'artist_name', 'date_year']]
df.rename(columns={'date_year': 'year'}, inplace=True)

# Save the results to a csv file
df.to_csv('data-out/musicbrainz/mb_FINAL_track_data.csv', index=False)

In [2]:
# Keep useful columns only
df.drop('track_id', axis=1, inplace=True)
df.drop('artist_id', axis=1, inplace=True)
df.drop('medium_id', axis=1, inplace=True)
df.drop('release_id', axis=1, inplace=True)

# Rename columns
df.rename(columns={'track_name': 'track'}, inplace=True)
df.rename(columns={'artist_name': 'artist'}, inplace=True)

In [3]:
# Filter for date_year = 2022
df_2022 = df[(df['year'] == 2021) | (df['year'] == 2022) | (df['year'] == 2023)]

In [4]:
# Remove duplicates
df_2022_clean = df_2022.drop_duplicates()

In [5]:
df_2022_clean

,track_name,artist_name,track_year
6993,For You,Bruce Springsteen,2022
6994,It’s Hard to Be a Saint in the City,Bruce Springsteen,2022
6995,Atlantic City,Bruce Springsteen,2022
6996,Straight Time,Bruce Springsteen,2022
6997,Tougher Than the Rest,Bruce Springsteen,2022
...,...,...,...
74191651,"Trio Sonata in G Major, BWV 1038: II. Vivace",Eryn Nicole,2022
74191652,"Trio Sonata in G Major, BWV 1038: III. Adagio",Eryn Nicole,2022
74191653,"Trio Sonata in G Major, BWV 1038: IV. Presto",Eryn Nicole,2022
74191654,"Herr Jesu Christ, dich zu uns wend, BWV 709 (T...",Eryn Nicole,2022


In [6]:
# Save the results to a csv file
df_2022_clean.to_csv('data-out/musicbrainz/extracted_track_data.csv', sep='\t', index=False)

### 2. Source : Wikipedia => Liste des albums sortis en 2022

  - #### <a href='https://en.wikipedia.org/wiki/List_of_2022_albums_(January%E2%80%93June)'>Janvier-Juin 2002</a>
  - #### <a href='https://en.wikipedia.org/wiki/List_of_2022_albums_(July%E2%80%93December)'>Juillet-Décembre 2002</a>
  
  - ### Scrapping des pages wikipedia référençant la listes des albums sortis en 2022

In [2]:
import requestsmb_data
#get page
urls = ['https://en.wikipedia.org/wiki/List_of_2022_albums_(January%E2%80%93June)','https://en.wikipedia.org/wiki/List_of_2022_albums_(July%E2%80%93December)']

#create list of artists and albums
artists = []
albums = []

for url in urls:
    page = requests.get(url)

    #create soup
    soup = BeautifulSoup(page.content, 'html.parser')

    #find all tables
    tables = soup.find_all('table', {'class':'wikitable plainrowheaders'})

    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 1: #Check if there are at least two elements in cells
                artists.append(cells[0].text[:-1])
                albums.append(cells[1].text[:-1])

#create dataframe
df = pd.DataFrame(list(zip(artists, albums)), columns =['artist', 'album']) 
df.to_csv('data-out/musicbrainz/albums_released_2022.csv', sep='\t', index=False)
print(df)

                          artist  \
0                       Up10tion   
1                         Kep1er   
2                          Onewe   
3                            ASP   
4                        Omega X   
...                          ...   
1508                      Weezer   
1509  YoungBoy Never Broke Again   
1510                   Sugababes   
1511                  DC the Don   
1512                  Zach Bryan   

                                                  album  
0                                               Novella  
1                                          First Impact  
2                                  Planet Nine: Voyager  
3                                               Placebo  
4                                          Love Me Like  
...                                                 ...  
1508                                       SZNZ: Winter  
1509                                         Lost Files  
1510                                     The 